# Source Data from PA

In [37]:
from TM1py.Services import TM1Service
from TM1py.Utils.Utils import element_names_from_element_unique_names, build_pandas_dataframe_from_cellset
from creds import tm1_credentials

with TM1Service(address=tm1_credentials["address"], port=tm1_credentials["port"], user=tm1_credentials["user"], password=tm1_credentials["password"], ssl=tm1_credentials["ssl"]) as tm1:
    data = tm1.cubes.cells.execute_view(cube_name='DMD Grid Planning', view_name='Default', private=False)
    allocation = build_pandas_dataframe_from_cellset(data, multiindex=False)
    data = tm1.cubes.cells.execute_view(cube_name='DMD Customer Demand', view_name='Default', private=False)
    customer = build_pandas_dataframe_from_cellset(data, multiindex=False)
    data = tm1.cubes.cells.execute_view(cube_name='DMD Transmission Capacity', view_name='Default', private=False)
    distribution = build_pandas_dataframe_from_cellset(data, multiindex=False)
    data = tm1.cubes.cells.execute_view(cube_name='}ElementAttributes_DMD Transmission Route', view_name='[Python] Route Optimization', private=False)
    distributionlocation = build_pandas_dataframe_from_cellset(data, multiindex=False)
    data = tm1.cubes.cells.execute_view(cube_name='}ElementAttributes_Customer', view_name='[Python] Route Optimization', private=False)
    customerlocation = build_pandas_dataframe_from_cellset(data, multiindex=False)

In [38]:
distributionlocation.set_index('DMD Transmission Route', inplace=True)
customerlocation.set_index('Customer', inplace=True)
customer.set_index('Customer', inplace=True)
distribution.set_index('DMD Transmission Route', inplace=True)

# Distance Calculation

In [39]:
allocation['Customer-Coords'] = allocation['Customer'].apply(lambda x: [float(coord) for coord in customerlocation.loc[x]['Values'].strip('(),').split(',')])
allocation['Distribution-Coords'] = allocation['DMD Transmission Route'].apply(lambda x: [float(coord) for coord in distributionlocation.loc[x]['Values'].strip('(),').split(',')])

In [40]:
from geopy.distance import geodesic

In [41]:
for index, row in allocation.iterrows():
    allocation.loc[index, 'Distance Difference'] = round(geodesic(row['Customer-Coords'], row['Distribution-Coords']).km,0)

# Create Indicies

In [42]:
allocation.set_index(['Customer', 'DMD Transmission Route', 'Time Date', 'Version'], inplace=True)
customer.reset_index(inplace=True)
customer.set_index(['Customer', 'Time Date', 'Version'], inplace=True)
distribution.reset_index(inplace=True)
distribution.set_index(['DMD Transmission Route', 'Time Date', 'Version'], inplace=True)

# Prepare Model

In [43]:
from docplex.mp.model import Model

In [44]:
# Model
mdl = Model(name='GTEng')

In [45]:
allocvars = mdl.continuous_var_dict(allocation.index)

In [46]:
for customer_key in customer.index.unique():
    mdl.add_constraint(mdl.sum(allocvars[allocvar] for allocvar in allocvars.keys() if (allocvar[0], allocvar[2], allocvar[3]) == customer_key) >= customer.loc[customer_key]['Values'])

In [47]:
for dist_key in distribution.index.unique():
    mdl.add_constraint(mdl.sum(allocvars[allocvar] for allocvar in allocvars.keys() if (allocvar[1], allocvar[2], allocvar[3]) == dist_key) <= distribution.loc[dist_key]['Values'])

# Solve Problem

In [48]:
# Minimize cost
obj = mdl.sum(val*allocation.loc[combo]['Distance Difference'] for combo, val in allocvars.items())
mdl.minimize(obj)

In [49]:
# Solve
ok = mdl.solve()

In [50]:
results = ok.get_value_dict(allocvars)

# Push Data Back to PA

In [51]:
cellset = {(key[2], key[3], key[0], key[1], 'Base', 'Amount'):value for key,value in results.items()}

In [52]:
with TM1Service(address=tm1_credentials["address"], port=tm1_credentials["port"], user=tm1_credentials["user"], password=tm1_credentials["password"], ssl=tm1_credentials["ssl"]) as tm1:
    tm1.cubes.cells.write_values('DMD Grid Planning', cellset, dimensions=['Time Date', 'Version', 'Customer', 'DMD Transmission Route', 'DMD Grid Planning Source', 'DMD Grid Planning Measure'])